In [1]:
require "hdf5"
require "optim"
require "nn"

In [25]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]
nfeatures = f:read("nfeatures"):all()[1]

--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

In [56]:
--Fits the count-based model with given smoothing parameters
--X : sequence features
--Y : sequence labels
--alpha1 : additive alpha for class counts
--alpha2 : additive alpha for class-conditional feature counts
function fit(X, Y, alpha1, alpha2)
    --count matrix of class transitions: p(y_i|y_{i-1},\theta)
    local C_trans = torch.ones(nclasses,nclasses)*alpha1
    --count matrix of class-conditional features: p(x_i|y_i,\theta)
    local C_emi = torch.ones(nclasses, nwords)*alpha2
    
    for i = 2,X:size(1) do 
        local y_curr = Y[i]
        local y_prev = Y[i-1]
        local x_curr = X[i]
        C_trans[y_prev][y_curr] = C_trans[y_prev][y_curr] + 1
        C_emi[y_curr][x_curr] = C_emi[y_curr][x_curr] + 1
    end
    return C_trans, C_emi
end

--Returns a distribution over the various tags
--y_prev : previous class tag
--x_curr : current feature 
--C_trans : transition count matrix
--C_emi : emission count matrix
function predict_distri(y_prev, x_curr, C_trans, C_emi)
    --compute transition and emission distributions
    local trans = C_trans[y_prev]/torch.sum(C_trans[y_prev])
    local emi = C_emi[{{},{x_curr,x_curr}}]/torch.sum(C_emi[{{},{x_curr,x_curr}}])
    return torch.log(trans) + torch.log(emi)
end

--Returns a log-probability table for a given sentence and labels
--sen : a single sentence with word features
--labels : corresponding target labels for a sentence
--C_trans : transition count matrix
--C_emi : emission count matrix
function proba_table(sen, labels, C_trans, C_emi)
    local table = torch.ones(X_valid_sen:size()[2],nclasses)
    for j=1,X_sen:size()[2] do
        local feat = X_sen[i][j]
        --end of sentence
        if feat == 0 then
            break
        end
        table[j] = predict_distri(2, feat, C_trans, C_emi)
    end
    return table:t()
end

In [5]:
ct, ce = fit(X_train, Y_train, 1, 1)

In [57]:
proba_table(X_valid_sen, _, ct, ce)

Columns 1 to 8
 -8.7271  -1.0891  -0.6341  -2.8067  -2.2189  -2.2189  -0.8208  -2.8067
 -8.9699  -3.8968  -6.6484  -3.0495  -3.1549  -3.1549  -4.5900  -3.0495
-16.1477 -11.0746 -13.8261 -10.2273 -10.3327 -10.3327 -11.7678 -10.2273
-16.1477 -11.0746 -13.8261 -10.2273 -10.3327 -10.3327 -11.7678 -10.2273
-15.4545 -10.3815 -13.1330  -9.5342  -9.6395  -9.6395 -11.0746  -9.5342
-16.1477 -11.0746 -13.8261 -10.2273 -10.3327 -10.3327 -11.7678 -10.2273
-16.1477 -11.0746 -13.8261 -10.2273 -10.3327 -10.3327 -11.7678 -10.2273
 -3.5898  -6.6320  -9.3835  -5.7847  -5.8900  -5.8900  -7.3251  -5.7847
-16.1477 -11.0746 -13.8261 -10.2273 -10.3327 -10.3327 -11.7678 -10.2273

Columns 9 to 16
 -1.7081  -2.8067  -2.8067  -0.6139   1.0000   1.0000   1.0000   1.0000
 -3.3372  -3.0495  -3.0495  -8.3725   1.0000   1.0000   1.0000   1.0000
-10.5150 -10.2273 -10.2273 -15.5503   1.0000   1.0000   1.0000   1.0000
-10.5150 -10.2273 -10.2273 -15.5503   1.0000   1.0000   1.0000   1.0000
 -9.8218  -9.5342  -9.5342 -14.8

000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000

Columns 57 to 63
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.0000   1.0000   1.0000
  1.0000   1.0000   1.0000   1.0000   1.00

In [26]:
print(X_valid_sen[1]:sub(1,15))

    1
 1964
  630
 8763
 6403
 1924
 1617
 8764
 2093
 8765
 8766
   11
    0
    0
    0
[torch.DoubleTensor of size 15]



In [31]:
X_valid_sen:size()[2]

63	


In [18]:
X_train:size()

 52616
[torch.LongStorage of size 1]



In [20]:
ce:size()

     9
 13428
[torch.LongStorage of size 2]

